In [1]:
import praw
import pandas as pd
import datetime
from praw.models import MoreComments
import nltk
import nltk
nltk.download('vader_lexicon')
import nltk.sentiment

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/timothyhansen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Reddit API authentication:

In [2]:
reddit = praw.Reddit(client_id='BxrF2g9d3egQ4g', \
                     client_secret='T-pkCmWCBxLjVlNZk88OjvcH7sk', \
                     user_agent='SDS2019')

## Download comments for movies on reddit with timestamp

In [18]:
def get_reddit_comments(movie_title, release_date, testing = True):
    subreddit = reddit.subreddit('movies')
    
    comments = []
    subid = []
    treeid = []
    date = []
    
    searchresults = subreddit.search(movie_title, sort = 'top')
    i = 0
    for subidx, submission in enumerate(searchresults):
        
        if testing:
            if i > 5:
                break
            i += 1
            
        try:
            comment_tree = submission.comments.list()
            for treeidx, top_lvl_comment in enumerate(comment_tree):
                
                # Erstat disse to linjer hvis i vil have 
                # data ud af "morecomments" objekterne
                if isinstance(top_lvl_comment, MoreComments):
                    continue
                
                subid.append(subidx)
                comments.append(top_lvl_comment.body)
                treeid.append(treeidx)
                date.append(top_lvl_comment.created_utc)
                
        except:
            pass
    df = pd.DataFrame({'submission': subid, 'comment_tree': treeid, 'comment': comments, 'date': date})
    df['date'] = pd.to_datetime(df['date'], unit = 's')
#    print(df.head)
#    print(df.dtypes)
    
    #mask = [(df['date'] >= start_date) & (df['date'] < end_date)]
    #df = df.loc[mask]
    df = df.set_index(['date'])
    
    end_date = datetime.datetime.strptime(release_date, '%Y-%m-%d') + datetime.timedelta(days = 60)
    df_before = df[:release_date]
    df_after = df[release_date:end_date]
    
    return df_before, df_after

In [19]:
#start_date = date(year = 2019, month = 6, day = 15)
#end_date = date(year = 2019, month = 8, day = 15)

#start_date = date(2019, 6, 15)
#end_date = date(2019, 8, 15)

once_upon_before, once_upon_after = get_reddit_comments('once upon a time in hollywood', '2019-07-26', testing = True)

#print(start_date)

KeyError: 1564099200000000000

In [17]:
datetime.datetime.strptime('2019-06-27', '%Y-%m-%d')

datetime.datetime(2019, 6, 27, 0, 0)

In [ ]:
once_upon

In [5]:
#print(once_upon)
pd.options.display.max_colwidth = 3000
once_upon['comment'].iloc[[3]]

3    That scene at the ranch when Cliff was going in to check on George Spahn was so fucking tense. Even the music from the TV was foreboding as hell.\n\nAnd holy shit, I didn't even realize that was Dakota Fanning. Totally didn't recognize her.
Name: comment, dtype: object

## Linguistic analysis of commments:
Output --> four mean scores

In [33]:
def sentiment(movie_df):
    vader = nltk.sentiment.vader.SentimentIntensityAnalyzer()
    vader_df = pd.DataFrame(list(movie_df['comment'].apply(vader.polarity_scores)))
    vader_mean = vader_df['compound'].mean()
    return vader_mean, len(vader_df)


In [32]:
len(vader_df)

12766

In [34]:
movie_list = ['phantom of the menace', 'attack of the clones', 'revenge of the sith', 'a new hope', 'empire strikes back', 'return of the jedi', 'the force awakens', 'the last jedi']
#movie_list = ['shazam', 'suicide squad']

sentiments = []
num_comments = []
for movie in movie_list:
    comments = get_reddit_comments(movie, testing = True)
    snt = sentiment(comments)
    sentiments.append(snt[0])
    num_comments.append(snt[1])

In [36]:
num_comments

[2173, 2334, 2888, 2964, 2860, 2886, 2923, 2968]